# DOG DISEASE DETECTION
## Tim : 
- Christian adi Ananta
- Ezra Abednego

### Dataset : 

### Metode : 




In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
!pip install tensorflow==2.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 24.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: keras
    Found existing

In [3]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [4]:
import urllib.request
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from shutil import copyfile
from PIL import Image 
import matplotlib.pyplot as plt
import random
from shutil import copyfile
from tensorflow.keras import regularizers

In [5]:
#Cek File Dataset
print("Number of cat images:",os.listdir('/kaggle/input/skin-disease-dogs/Skin_Disease_Augment'))

Number of cat images: ['earinfection', 'dermatitis', 'skintumors', 'fleas', 'dandruff', 'ringworm']


In [6]:
#Membuat Folder Train Dan Validation
from pathlib import Path
path_img='/kaggle/input/skin-disease-dogs/Skin_Disease_Augment'
img_directory=os.listdir(path_img)
work_path = Path('/kaggle/working/')
for i in ["train","validation"]:
    work_path_parent = Path('/kaggle/working/'+i)
    work_path_parent.mkdir(exist_ok=True)
    for j in img_directory:
        work_path = Path('/kaggle/working/'+i+"/"+j)
        work_path.mkdir(exist_ok=True)

In [7]:
# Split Data dengan presentase 90% Train dan 10% Validation
def split_data(split_size):
    files=[]
    for file_dir in img_directory:
        name_image=[]
        for filename in os.listdir(path_img+"/"+file_dir):
            file=path_img+"/"+file_dir+"/"+filename
            if os.path.getsize(file) > 0:
                name_image.append(filename)
            else:
                print(filename + " is zero length, so ignoring.")
        files.append(name_image)
    for index,path_image in enumerate(files):
        train_len=int(len(path_image)*split_size)
        shuffle=random.sample(path_image,len(path_image))
        train_data=path_image[:train_len]
        validation_data=path_image[train_len:]
        for filename in train_data:
            file=path_img+"/"+img_directory[index]+"/"+filename
            destination="/kaggle/working/train/"+img_directory[index]+"/"+filename
            copyfile(file,destination)
        for filename in validation_data:
            file=path_img+"/"+img_directory[index]+"/"+filename
            destination="/kaggle/working/validation/"+img_directory[index]+"/"+filename
            copyfile(file,destination)
        
split_size=.8
split_data(split_size)

In [8]:
# Melakukan preprocessing data image dari directory

TRAINING_DIR = "/kaggle/working/train"
# Experiment with your own parameters to reach 99.9% validation accuracy or better
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   horizontal_flip=True,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))
VALIDATION_DIR = "/kaggle/working/validation/"

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=32,
                                                              class_mode='categorical',
                                                              target_size=(150, 150))

Found 3599 images belonging to 6 classes.
Found 901 images belonging to 6 classes.


In [9]:
# data_augmentation = tf.keras.Sequential([
#   tf.keras.layers.RandomFlip('horizontal'),
#   tf.keras.layers.RandomRotation(0.2),
# ])

# preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [10]:
pre_trained_model =  tf.keras.applications.InceptionV3(input_shape=(150,150,3),
                                                       include_top=False,
                                                       weights='imagenet')

87910968/87910968 [==============================] - 0s 0us/step


In [11]:
#freeze layer
for layer in pre_trained_model.layers:
    layer.trainable = False

In [12]:
#get last layer
last_layer =  pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [13]:
#Add Dense Layer
x = layers.Flatten()(last_output)
x = layers.Dense(64,activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(6, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 74, 74, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 74, 74, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 74, 74, 32)           0         ['batch_normalization[0][0

In [14]:
#Train Model
model.fit(train_generator,
                  epochs=50,
                  validation_data=validation_generator)

Epoch 1/50
113/113 [==============================] - 73s 611ms/step - loss: 1.0914 - accuracy: 0.5924 - precision: 0.8115 - recall: 0.3695 - val_loss: 0.6643 - val_accuracy: 0.7680 - val_precision: 0.8591 - val_recall: 0.6226
Epoch 2/50
113/113 [==============================] - 66s 586ms/step - loss: 0.6580 - accuracy: 0.7685 - precision: 0.8480 - recall: 0.6710 - val_loss: 0.5157 - val_accuracy: 0.8269 - val_precision: 0.8864 - val_recall: 0.7358
Epoch 3/50
113/113 [==============================] - 67s 593ms/step - loss: 0.5075 - accuracy: 0.8208 - precision: 0.8756 - recall: 0.7572 - val_loss: 0.4252 - val_accuracy: 0.8602 - val_precision: 0.9141 - val_recall: 0.8147
Epoch 4/50
113/113 [==============================] - 66s 586ms/step - loss: 0.4192 - accuracy: 0.8527 - precision: 0.8943 - recall: 0.8091 - val_loss: 0.3321 - val_accuracy: 0.8846 - val_precision: 0.9195 - val_recall: 0.8491
Epoch 5/50
113/113 [==============================] - 70s 622ms/step - loss: 0.3614 - accura

In [15]:
class_map = train_generator.class_indices
classes_disease = []
for key in class_map.keys():
    classes_disease.append(key)
print(classes_disease)

['dandruff', 'dermatitis', 'earinfection', 'fleas', 'ringworm', 'skintumors']


In [16]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

for i in os.listdir('/kaggle/working/validation'):
    for j in os.listdir('/kaggle/working/validation/'+i):
        path='/kaggle/working/validation/'+i+'/'+j
        img = load_img(path, target_size=(150, 150))
        x = img_to_array(img)
        x /= 255
        x = np.expand_dims(x, axis=0)
        image_tensor = np.vstack([x])
        classes = model.predict(image_tensor)
        print(j)
        print(classes[0])
        print(classes_disease[np.argmax(classes[0])])
        break

        
# path = '/kaggle/working/validation/ringworm/ringworm_dark_40.png'
# img = load_img(path, target_size=(150, 150))
# x = img_to_array(img)
# x /= 255
# x = np.expand_dims(x, axis=0)
# image_tensor = np.vstack([x])
# classes = model.predict(image_tensor)
# print(classes[0])
# print(np.argmax(classes[0]))
# print(classes_disease[np.argmax(classes[0])])

1/1 [==============================] - 1s 999ms/step
skintumors_light_87.png
[9.7135471e-06 3.0353388e-03 2.8565987e-07 9.3844022e-08 5.2641798e-07
 9.9695408e-01]
skintumors
1/1 [==============================] - 0s 40ms/step
lickgranuloma9 - Copy (3).png
[7.3659851e-07 9.8072433e-01 7.0063634e-06 3.5729298e-07 7.4718682e-06
 1.9260081e-02]
dermatitis
1/1 [==============================] - 0s 42ms/step
earinfection_flipshori_30.png
[1.0522302e-06 6.2884201e-06 9.9998879e-01 1.7963300e-10 3.0887357e-08
 3.7792299e-06]
earinfection
1/1 [==============================] - 0s 40ms/step
dandruff_light_11.png
[9.5621723e-01 1.9790747e-08 2.5732660e-08 4.3782759e-02 1.1347650e-08
 1.4268061e-09]
dandruff
1/1 [==============================] - 0s 40ms/step
fleas_flips_267.png
[6.6062817e-03 1.2050000e-04 7.5346411e-06 9.8748487e-01 5.2599269e-03
 5.2082987e-04]
fleas
1/1 [==============================] - 0s 39ms/step
ringworm5 - Copy (4).png
[1.46846205e-05 6.86454645e-04 4.75989964e-07 4.633

In [17]:
model.save("model_saved.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
